In [70]:
import pandas as pd
from pandas_profiling import ProfileReport
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sweetviz as sv
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format
from IPython.core.display import display,HTML
display(HTML("<style>.container {width: 98% !important; }</style>"))
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [10,10]
pd.set_option('display.max_columns' ,500)
pd.set_option('display.max_rows' ,500)

In [121]:
data = pd.read_excel('A:\Wells Fargo\Learning and Development\ML Stack\ML Case Study.xlsx')
#data.dtypes

In [94]:
data.head()

,Experience,Income,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_2,Education_3,Family_2,Family_3,Family_4
0,1,49,1.60,0,0,1,0,0,0,0,0,0,0,1
1,19,34,1.50,0,0,1,0,0,0,0,0,0,1,0
2,15,11,1.00,0,0,0,0,0,0,0,0,0,0,0
3,9,100,2.70,0,0,0,0,0,0,1,0,0,0,0
4,8,45,1.00,0,0,0,0,0,1,1,0,0,0,1


In [131]:
#pd.pivot_table(data,index='education',columns='Personal Loan')
data.groupby(['Personal Loan']).mean()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Securities Account,CD Account,Online,CreditCard
Personal Loan,,,,,,,,,,,,,
0,2512.17,45.37,20.13,66.24,93152.43,2.37,1.73,1.84,51.79,0.10,0.04,0.60,0.29
1,2390.65,45.07,19.84,144.75,93153.20,2.61,3.91,2.23,100.85,0.12,0.29,0.61,0.30


In [91]:
#Number of unique in each column
data.nunique()

ID                    5000
Age                     45
Experience              47
Income                 162
ZIP Code               467
Family                   4
CCAvg                  108
Education                3
Mortgage               347
Personal Loan            2
Securities Account       2
CD Account               2
Online                   2
CreditCard               2
dtype: int64

In [89]:
#Number of people with zero mortgage
len(data[data.Mortgage == 0])

3462

In [90]:
#Number of people with zero credit card spending per month
len(data[data.CCAvg == 0])

106

In [120]:
#Value counts of all categorical columns
print(data['CreditCard'].value_counts(normalize=True).sort_index())

0   0.71
1   0.29
Name: CreditCard, dtype: float64


In [72]:
#Making Education and Family categorical variables as dummy variables. Other categorical variables have only 2 possible values - 0/1
data = pd.get_dummies(data,columns=['Education','Family'],drop_first=True)

In [76]:
#Experimenting with grouping ZIP codes
#print(int(data['ZIP Code'].values[0]/100))
#data['ZIP Code'].to_csv('zip_code.csv')
#new_zip = []
#for i in data['ZIP Code'].values :
#    new_zip.append(int(i/100))
#new_zip
#data['ZIP Code'] = new_zip
#data['ZIP Code'].nunique()

In [21]:
Profile = ProfileReport(data)
Profile.to_file("Data_Profile_Report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 55.84it/s]


In [75]:
data.columns

Index(['Experience', 'Income', 'CCAvg', 'Mortgage', 'Personal Loan',
       'Securities Account', 'CD Account', 'Online', 'CreditCard',
       'Education_2', 'Education_3', 'Family_2', 'Family_3', 'Family_4'],
      dtype='object')

In [74]:
data.drop(columns=['ID','Age','ZIP Code'],inplace=True)

In [76]:
vif = pd.DataFrame()
vif["feature"] = data.columns
vif["VIF"] = [variance_inflation_factor(data.values,i) for i in range(data.shape[1])]
vif

,feature,VIF
0,Experience,3.17
1,Income,6.32
2,CCAvg,3.84
3,Mortgage,1.37
4,Personal Loan,1.68
5,Securities Account,1.26
6,CD Account,1.52
7,Online,2.35
8,CreditCard,1.53
9,Education_2,1.63


In [45]:
datacent = pd.DataFrame()
for i in range(len(data.columns)-1) :
    datacent[data.columns[i]] =  data[data.columns[i]].values - np.mean(data[data.columns[i]].values)  
datacent[data.columns[len(data.columns)-1]] = data[data.columns[len(data.columns)-1]].values
vif["VIF after centralizing"] = [variance_inflation_factor(datacent.values,i) for i in range(datacent.shape[1])]

In [46]:
vif

,feature,VIF,VIF after centralizing
0,Experience,3.44,1.01
1,Income,6.23,2.21
2,Family,4.10,1.06
3,CCAvg,3.84,1.73
4,Education,4.44,1.12
5,Mortgage,1.37,1.05
6,Personal Loan,1.61,1.62
7,Securities Account,1.26,1.14
8,CD Account,1.53,1.40
9,Online,2.43,1.04


In [11]:
data.corr()

,Experience,Income,CCAvg,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard,Education_2,Education_3,Family_2,Family_3,Family_4
Experience,1.00,-0.05,-0.05,-0.01,-0.01,-0.00,0.01,0.01,0.01,-0.02,0.02,0.01,0.03,-0.07
Income,-0.05,1.00,0.65,0.21,0.50,-0.00,0.17,0.01,-0.00,-0.13,-0.11,0.13,-0.08,-0.14
CCAvg,-0.05,0.65,1.00,0.11,0.37,0.02,0.14,-0.00,-0.01,-0.09,-0.08,0.10,-0.07,-0.09
Mortgage,-0.01,0.21,0.11,1.00,0.14,-0.01,0.09,-0.01,-0.01,-0.03,-0.01,0.04,-0.01,-0.02
Personal Loan,-0.01,0.50,0.37,0.14,1.00,0.02,0.32,0.01,0.00,0.07,0.09,-0.03,0.06,0.03
Securities Account,-0.00,-0.00,0.02,-0.01,0.02,1.00,0.32,0.01,-0.02,0.01,-0.01,-0.00,0.00,0.02
CD Account,0.01,0.17,0.14,0.09,0.32,0.32,1.00,0.18,0.28,0.01,0.01,-0.02,0.04,-0.01
Online,0.01,0.01,-0.00,-0.01,0.01,0.01,0.18,1.00,0.00,0.02,-0.02,-0.02,0.01,0.01
CreditCard,0.01,-0.00,-0.01,-0.01,0.00,-0.02,0.28,0.00,1.00,-0.01,-0.00,0.02,-0.01,0.01
Education_2,-0.02,-0.13,-0.09,-0.03,0.07,0.01,0.01,0.02,-0.01,1.00,-0.41,-0.10,0.11,0.09


In [77]:
x = data.drop(columns='Personal Loan')
y = data['Personal Loan']

In [78]:
Algo = []
train = []
test = []
recall = []
roc = []

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score,roc_auc_score

In [80]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=5)

In [81]:
#Logistic Regression
classifier = LogisticRegression()
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Logistic Regression')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [82]:
#Naive Bayes
classifier = GaussianNB()
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Naive Bayes')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [83]:
#SVM
classifier = SVC()
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('SVM')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [84]:
#KNearest Neighbors
classifier = KNeighborsClassifier()
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('K Nearest Neighbors')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [85]:
#Decision Tree with gini
classifier = DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=5)
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Decision Tree with gini')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [86]:
#Decision Tree with entropy
classifier = DecisionTreeClassifier(criterion='entropy',max_depth=3,random_state=5)
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Decision Tree with entropy')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [87]:
#Random Forest
classifier = RandomForestClassifier(criterion='entropy',n_estimators=100,random_state=5,bootstrap=True)
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Random Forest')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [88]:
#Bagging Classifier
classifier = BaggingClassifier(n_estimators=100,max_samples=0.7,bootstrap=True,oob_score=True,random_state=11)
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Bagging Classifier')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [89]:
#Adaboost Classifier
classifier = AdaBoostClassifier(n_estimators=200,learning_rate=0.1,random_state=11)
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('Adaboost')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [90]:
#GradientBoost
classifier = GradientBoostingClassifier(n_estimators=200,random_state=11)
classifier.fit(xtrain,ytrain)
ypredict = classifier.predict(xtest)
Algo.append('GradientBoost')
train.append(classifier.score(xtrain,ytrain))
test.append(classifier.score(xtest,ytest))
recall.append(recall_score(ypredict,ytest))
roc.append(roc_auc_score(ypredict,ytest))

In [91]:
results = pd.DataFrame()
roc_w = 0.5
recall_w = 0.2
test_w = 0.2
train_w = 0.1
results['Model'] = Algo
results['Training Accuracy'] = train
results['Testing Accuracy'] = test
results['Recall'] = recall
results['ROC AUC Score'] = roc
results['Net Score'] = [roc_w*roc[i]+recall_w*recall[i]+test_w*test[i]+train_w*train[i] for i in range(len(Algo))]
results = results.set_index('Model')
#results
results.sort_values(by=['Net Score'],ascending=False)

,Training Accuracy,Testing Accuracy,Recall,ROC AUC Score,Net Score
Model,,,,,
Random Forest,1.00,0.99,0.99,0.99,0.99
GradientBoost,1.00,0.99,0.96,0.98,0.98
Bagging Classifier,1.00,0.98,0.96,0.97,0.97
Adaboost,0.97,0.96,0.94,0.95,0.95
Logistic Regression,0.96,0.96,0.93,0.94,0.95
Decision Tree with gini,0.97,0.96,0.91,0.94,0.94
Decision Tree with entropy,0.96,0.95,0.81,0.89,0.89
SVM,0.91,0.90,0.58,0.75,0.76
K Nearest Neighbors,0.94,0.90,0.51,0.72,0.74


In [92]:
#Selected Model: Random Forest
import pickle
classifier = RandomForestClassifier(criterion='entropy',n_estimators=100,random_state=5,bootstrap=True)
classifier.fit(xtrain,ytrain)
pickle.dump(classifier, open('model.pkl','wb'))

In [93]:
x.head()

,Experience,Income,CCAvg,Mortgage,Securities Account,CD Account,Online,CreditCard,Education_2,Education_3,Family_2,Family_3,Family_4
0,1,49,1.60,0,1,0,0,0,0,0,0,0,1
1,19,34,1.50,0,1,0,0,0,0,0,0,1,0
2,15,11,1.00,0,0,0,0,0,0,0,0,0,0
3,9,100,2.70,0,0,0,0,0,1,0,0,0,0
4,8,45,1.00,0,0,0,0,1,1,0,0,0,1


In [101]:
print(classifier.predict([[1,	49,	1.60,	0,	1,	0,	0,	0	,0,	0,	0,	0,	1]]))

[0]


In [98]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Personal Loan, dtype: int64